In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import *
import pandas as pd
import pyspark.pandas as ps

/home4/tc737978/Documents/M2/Machine Learning/TD/TD1/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Initialisation de Spark

In [3]:
spark = SparkSession.builder.appName("Projet InfoD").config("spark.jars", "/home1/tc737978/Documents/M2/Info décisionnelle/postgresql-42.7.5.jar").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)


25/02/20 13:24:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Chargement des données

In [4]:
tip = spark.read.csv("yelp_academic_dataset_tip.csv", header=True, inferSchema=True)
business_source = pd.read_json("yelp_academic_dataset_business.json", lines=True)
checkin = spark.read.json("yelp_academic_dataset_checkin.json")
reviews = spark.read.jdbc("jdbc:postgresql://stendhal:5432/tpid2020", "yelp.review", properties={"user": "tpid", "password": "tpid", "driver": "org.postgresql.Driver"})

In [4]:
tip.show()

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|UYX5zL_Xj9WEc_Wp-...|               0|2013-11-26 18:20:08|Here for a quick mtg|hf27xTME3EiCp6NL6...|
|Ch3HkwQYv1YKw_FO0...|               0|2014-06-15 22:26:45|Cucumber strawber...|uEvusDwoSymbJJ0au...|
|rDoT-MgxGRiYqCmi0...|               0|2016-07-18 22:03:42|Very nice good se...|AY-laIws3S7YXNl_f...|
|OHXnDV01gLokiX1EL...|               0|2014-06-06 01:10:34|It's a small plac...|Ue_7yUlkEbX4AhnYd...|
|GMrwDXRlAZU2zj5nH...|               0|2011-04-08 18:12:01|8 sandwiches, $24...|LltbT_fUMqZ-ZJP-v...|
|ALwAlxItASeEs2vYA...|               0|2015-05-20 20:17:38|Great ramen! Not ...|HHNBqfbDR8b1iq-QG...|
|d_L-rfS1vT3JMzgCU...|               0|2014-09-01 01:23:48|Cochinita Pibil w...|r0

In [5]:
business_source = spark.createDataFrame(business_source)
business_source.show()

25/02/20 13:25:11 WARN TaskSetManager: Stage 3 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+---------------+-----+-----------+-------------+---------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|           city|state|postal_code|     latitude|      longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+---------------+-----+-----------+-------------+---------------+-----+------------+-------+--------------------+--------------------+--------------------+
|f9NumwFMBDn751xgF...|The Range At Lake...|     10913 Bailey Rd|      Cornelius|   NC|      28031|   35.4627242|    -80.8526119|  3.5|          36|      1|{BikeParking -> T...|Active Life, Gun/...|{Thursday -> 11:0...|
|Yzvjg0SayhoZgCljU...|   Carlos Santo, NMD|8880 E Via Linda,...|     Scottsdale|   AZ|      85258|   33.5694041|   -111.8902

In [5]:
checkin.show()

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|--1UhMGODdWsrMast...|2016-04-26 19:49:...|
|--6MefnULPED_I942...|2011-06-04 18:22:...|
|--7zmmkVg-IMGaXbu...|2014-12-29 19:25:...|
|--8LPVSo5i0Oo61X0...| 2016-07-08 16:43:30|
|--9QQLMTbFzLJ_oT-...|2010-06-26 17:39:...|
|--9e1ONYQuAa-CB_R...|2010-02-08 05:56:...|
|--DaPTJW3-tB1vP-P...|2012-06-03 17:46:...|
|--DdmeR16TRb3LsjG...|2012-11-02 21:26:...|
|--EF5N7P70J_UYBTP...|2018-05-25 19:52:...|
|--EX4rRznJrltyn-3...|2010-02-26 17:05:...|
|--FBCX-N37CMYDfs7...|2010-05-31 07:57:...|
|--FLdgM0GNpXVMn74...|2012-10-23 18:47:...|
|--FnvijzY20d1nk9H...|2019-09-19 02:42:...|
|--GM_ORV2cYS-h38D...|2011-09-11 18:16:...|
|--I7YYLada0tSLkOR...|2014-11-07 00:51:...|
|--KCl2FvVQpvjzmZS...|2011-07-29 16:53:...|
|--KQsXc-clkO7oHRq...|2010-05-02 23:57:...|
|--Ni3oJ4VOqfOEu7S...| 2019-06-07 17:54:58|
|--Rsj71PBe31h5Ylj...|2011-12-15 18:09:...|
|--S62v0QgkqQaVUhF...|2010-12-25

In [ ]:
reviews.show()

## Création des dataframes

### Business

In [7]:
Business = business_source.select("business_id")
Business = Business.withColumn("ID_Categorie", monotonically_increasing_id())
Business = Business.withColumn("ID_Attributes", monotonically_increasing_id())
Business = Business.withColumn("ID_Hours", monotonically_increasing_id())
Business = Business.withColumn("ID_Localisation", monotonically_increasing_id())
Business = Business.withColumn("ID_Reviews", monotonically_increasing_id())
Business.show()


+--------------------+------------+-------------+--------+---------------+----------+
|         business_id|ID_Categorie|ID_Attributes|ID_Hours|ID_Localisation|ID_Reviews|
+--------------------+------------+-------------+--------+---------------+----------+
|f9NumwFMBDn751xgF...|           0|            0|       0|              0|         0|
|Yzvjg0SayhoZgCljU...|           1|            1|       1|              1|         1|
|XNoUzKckATkOD1hP6...|           2|            2|       2|              2|         2|
|6OAZjbxqM5ol29BuH...|           3|            3|       3|              3|         3|
|51M2Kk903DFYI6gnB...|           4|            4|       4|              4|         4|
|cKyLV5oWZJ2NudWgq...|           5|            5|       5|              5|         5|
|oiAlXZPIFm2nBCt0D...|           6|            6|       6|              6|         6|
|ScYkbYNkDgCneBrD9...|           7|            7|       7|              7|         7|
|pQeaRpvuhoEqudo3u...|           8|            8|     

25/02/20 13:25:40 WARN TaskSetManager: Stage 4 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Localisation

In [6]:
Localisation = business_source.select("address", "city", "state", "postal_code", "latitude", "longitude")
Localisation = Localisation.withColumn("id", monotonically_increasing_id())
Localisation = Localisation.withColumn("country", \
                        when(Localisation["state"] == "NC", "USA") \
                        .when(Localisation["state"] == "AZ", "USA") \
                        .when(Localisation["state"] == "QC", "CAN") \
                        .when(Localisation["state"] == "NV", "USA") \
                        .when(Localisation["state"] == "IL", "USA") \
                        .when(Localisation["state"] == "ON", "CAN") \
                        .when(Localisation["state"] == "AB", "CAN") \
                        .when(Localisation["state"] == "PA", "USA") \
                        .when(Localisation["state"] == "WI", "USA") \
                        .when(Localisation["state"] == "SC", "USA") \
                        .when(Localisation["state"] == "OH", "USA") \
                        .when(Localisation["state"] == "CA", "USA") \
                        .when(Localisation["state"] == "TX", "USA") \
                        .when(Localisation["state"] == "NY", "USA") \
                        .when(Localisation["state"] == "CO", "USA") \
                        .when(Localisation["state"] == "XWY", "USA") \
                        .when(Localisation["state"] == "GA", "USA") \
                        .when(Localisation["state"] == "BC", "CAN") \
                        .when(Localisation["state"] == "YT", "CAN") \
                        .when(Localisation["state"] == "HPL", "USA") \
                        .when(Localisation["state"] == "AL", "USA") \
                        .when(Localisation["state"] == "UT", "USA") \
                        .when(Localisation["state"] == "VT", "USA") \
                        .when(Localisation["state"] == "WA", "USA") \
                        .when(Localisation["state"] == "NE", "USA") \
                        .when(Localisation["state"] == "DOW", "USA") \
                        .when(Localisation["state"] == "MI", "USA") \
                        .when(Localisation["state"] == "FL", "USA") \
                        .when(Localisation["state"] == "AR", "USA") \
                        .when(Localisation["state"] == "HI", "USA") \
                        .when(Localisation["state"] == "MB", "CAN") \
                        .when(Localisation["state"] == "OR", "USA") \
                        .when(Localisation["state"] == "AK", "USA") \
                        .when(Localisation["state"] == "VA", "USA") \
                        .when(Localisation["state"] == "CT", "USA") \
                        .when(Localisation["state"] == "MO", "USA") \
                        .when(Localisation["state"] == "DUR", "USA") \
                        .otherwise("Unknown"))
Localisation.show()

+--------------------+---------------+-----+-----------+-------------+---------------+---+-------+
|             address|           city|state|postal_code|     latitude|      longitude| id|country|
+--------------------+---------------+-----+-----------+-------------+---------------+---+-------+
|     10913 Bailey Rd|      Cornelius|   NC|      28031|   35.4627242|    -80.8526119|  0|    USA|
|8880 E Via Linda,...|     Scottsdale|   AZ|      85258|   33.5694041|   -111.8902637|  1|    USA|
|3554 Rue Notre-Da...|       Montreal|   QC|    H4C 1P4|    45.479984|      -73.58007|  2|    CAN|
|      1015 Sharp Cir|North Las Vegas|   NV|      89030|   36.2197281|   -115.1277255|  3|    USA|
|  4827 E Downing Cir|           Mesa|   AZ|      85205|   33.4280652|   -111.7266485|  4|    USA|
|1720 W Elliot Rd,...|        Gilbert|   AZ|      85233|   33.3503993|   -111.8271417|  5|    USA|
|6870 S Rainbow Bl...|      Las Vegas|   NV|      89118|   36.0639767|    -115.241463|  6|    USA|
| 6910 E S

25/02/20 13:10:22 WARN TaskSetManager: Stage 5 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Reviews

In [10]:
Reviews = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema=business_source.schema)

### Year

### Categories

In [7]:
Categories = Business.select("ID_Categorie")
Categories.show()


+------------+
|ID_Categorie|
+------------+
|           0|
|           1|
|           2|
|           3|
|           4|
|           5|
|           6|
|           7|
|           8|
|           9|
|          10|
|          11|
|          12|
|          13|
|          14|
|          15|
|          16|
|          17|
|          18|
|          19|
+------------+
only showing top 20 rows



25/02/20 13:10:39 WARN TaskSetManager: Stage 6 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Categories_List

In [8]:
# Sélectionner les colonnes nécessaires
categories_df = business_source.select("business_id", "categories")

# Transformer la colonne categories en liste
categories_df = categories_df.withColumn("categories_list", split(categories_df["categories"], ", "))

# Exploser la liste des catégories pour obtenir une ligne par catégorie
categories_list_df = categories_df.withColumn("category", explode(categories_df["categories_list"]))

# Joindre avec le DataFrame Business pour obtenir les ID_Categorie
categories_list_df = categories_list_df.join(Business, on="business_id")

# Sélectionner les colonnes finales et renommer
Categories_List = categories_list_df.select(col("ID_Categorie"), col("category").alias("Name"))

# Afficher le DataFrame final
Categories_List.show()


25/02/20 13:10:42 WARN TaskSetManager: Stage 7 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.
25/02/20 13:10:44 WARN TaskSetManager: Stage 8 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+------------+--------------------+
|ID_Categorie|                Name|
+------------+--------------------+
|          12|            Desserts|
|          12|                Food|
|          12|Ice Cream & Froze...|
|         219|            Shopping|
|         219|     Decks & Railing|
|         219|Paint-Your-Own Po...|
|         219|       Arts & Crafts|
|         219|           Cabinetry|
|         219|       Home Services|
|         219|          Carpenters|
|         219|         Contractors|
|         737|Garage Door Services|
|         737|       Home Services|
|         758|            Caterers|
|         758|      Specialty Food|
|         758|Event Planning & ...|
|         758|      American (New)|
|         758|         Food Trucks|
|         758|American (Traditi...|
|         758|         Restaurants|
+------------+--------------------+
only showing top 20 rows



### Attributes

In [8]:
Attributes = Business.select("ID_Attributes")
Attributes.show()

+-------------+
|ID_Attributes|
+-------------+
|            0|
|            1|
|            2|
|            3|
|            4|
|            5|
|            6|
|            7|
|            8|
|            9|
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
+-------------+
only showing top 20 rows



25/02/20 13:25:50 WARN TaskSetManager: Stage 5 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Attributes_List

In [14]:
#Sélectionner les colonnes nécessaires
attributes_df = business_source.select("business_id", "attributes")
attributes_df.show()
attributes_df.printSchema()

+--------------------+--------------------+
|         business_id|          attributes|
+--------------------+--------------------+
|f9NumwFMBDn751xgF...|{BikeParking -> T...|
|Yzvjg0SayhoZgCljU...|{ByAppointmentOnl...|
|XNoUzKckATkOD1hP6...|                NULL|
|6OAZjbxqM5ol29BuH...|{BikeParking -> T...|
|51M2Kk903DFYI6gnB...|{ByAppointmentOnl...|
|cKyLV5oWZJ2NudWgq...|{BusinessAcceptsC...|
|oiAlXZPIFm2nBCt0D...|{BikeParking -> T...|
|ScYkbYNkDgCneBrD9...|{ByAppointmentOnl...|
|pQeaRpvuhoEqudo3u...|{BikeParking -> F...|
|EosRKXIGeSWFYWwpk...|{BikeParking -> F...|
|MbZMmwo-eL0Jnm_Yb...|                NULL|
|7Dv4_HAxsxvadEsT5...|{BusinessAcceptsC...|
|M_guz7Dj7hX0evS67...|{BikeParking -> T...|
|JjJs3o60uQCfctDjs...|{ByAppointmentOnl...|
|kOICO53wbOiOJcKuC...|{ByAppointmentOnl...|
|rqcOZePlVvJP9Etzl...|{BusinessAcceptsC...|
|uZuh51lXu7tsrC8RA...|{BikeParking -> T...|
|nIEhsGbw0vJuYl05b...|{RestaurantsDeliv...|
|edQoeeBFUTYGwnUSE...|{BusinessAcceptsC...|
|Vwo64kNYDjKi98gUU...|{BikeParki

25/02/20 13:31:31 WARN TaskSetManager: Stage 6 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


In [16]:
# Convertir la map en colonnes
attributes_keys = attributes_df.select("business_id", "attributes.key")
attributes_values = attributes_df.select("business_id", "attributes.value")
attributes_keys.show()
attributes_values.show()

+--------------------+----+
|         business_id| key|
+--------------------+----+
|f9NumwFMBDn751xgF...|NULL|
|Yzvjg0SayhoZgCljU...|NULL|
|XNoUzKckATkOD1hP6...|NULL|
|6OAZjbxqM5ol29BuH...|NULL|
|51M2Kk903DFYI6gnB...|NULL|
|cKyLV5oWZJ2NudWgq...|NULL|
|oiAlXZPIFm2nBCt0D...|NULL|
|ScYkbYNkDgCneBrD9...|NULL|
|pQeaRpvuhoEqudo3u...|NULL|
|EosRKXIGeSWFYWwpk...|NULL|
|MbZMmwo-eL0Jnm_Yb...|NULL|
|7Dv4_HAxsxvadEsT5...|NULL|
|M_guz7Dj7hX0evS67...|NULL|
|JjJs3o60uQCfctDjs...|NULL|
|kOICO53wbOiOJcKuC...|NULL|
|rqcOZePlVvJP9Etzl...|NULL|
|uZuh51lXu7tsrC8RA...|NULL|
|nIEhsGbw0vJuYl05b...|NULL|
|edQoeeBFUTYGwnUSE...|NULL|
|Vwo64kNYDjKi98gUU...|NULL|
+--------------------+----+
only showing top 20 rows

+--------------------+-----+
|         business_id|value|
+--------------------+-----+
|f9NumwFMBDn751xgF...| NULL|
|Yzvjg0SayhoZgCljU...| NULL|
|XNoUzKckATkOD1hP6...| NULL|
|6OAZjbxqM5ol29BuH...| NULL|
|51M2Kk903DFYI6gnB...| NULL|
|cKyLV5oWZJ2NudWgq...| NULL|
|oiAlXZPIFm2nBCt0D...| NULL|
|ScYkbYNkDgC

25/02/20 13:40:34 WARN TaskSetManager: Stage 7 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.
25/02/20 13:40:34 WARN TaskSetManager: Stage 8 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Hours

In [12]:
Hours = business_source.select("hours.*")
Hours = Hours.withColumn("id", monotonically_increasing_id())
Hours.show()

AnalysisException: Can only star expand struct data types. Attribute: `ArrayBuffer(hours)`.